# First EDA - Team 19

EDA of the followin stores on ReclameAqui (27-03-2020 first 5 pages):
* tokestok-loja-online 
* tokestok 
* madeiramadeira 
* mobly 
* etna-home-store-loja-online 
* etna-home-store 
* casas-bahia-loja-online 
* casas-bahia-lojas-fisicas

## What questions will I be able to answer with this data?
Think about your data in terms of **achieving the objectives** we laid out in previous weeks.  It's also important to **outline insights and possibilities for future action** found in the data.

From our MVP target:
- Focus on answering customers' most frequent questions/problems
- Stores ranking
- Segmented analysis both online vs. bricks and mortar retailers
- Deliver a general snapshot of the vertical

## Reclame Aqui (RA) dataset sample

In [1]:
from util import agg_jsonls  # helper function
from pprint import pprint

# Unique reviews
unique_ids = set()
reviewsRA = []
for complaint in agg_jsonls("."):
    if "review_ID" in complaint and complaint["review_ID"] not in unique_ids:
        unique_ids.add(complaint["review_ID"])
        reviewsRA.append(complaint)
print(f"We have a total of {len(reviewsRA)} reviews!")

We have a total of 9681 reviews!


In [2]:
# Get an example of responses > 0
for review in reviewsRA:
    if "responses" in  review and len(review["responses"]["business"]) > 0:
        pprint(review)
        break

{'business_name': 'Inovar Móveis e Colchões',
 'city': 'São José dos Campos',
 'datetime': '2019-03-28T19:32:00Z',
 'description': 'Eu comprei uma mesa no dia 29.01.2019 a vista  e o prazo de '
                'entrega era no dia 15.03 e eles não entregaram e eu pedi do '
                'cancelamento do pedido no dia 22.03 e até agora na me '
                'estornaram o valor que eu paguei do produto',
 'responses': {'business': [{'content': 'Boa tarde Sr Francisca, já estamos '
                                        'verificando o ocorrido e '
                                        'solicitaremos ao setor de logística '
                                        'para programarem a sua entrega. Att, '
                                        'Inovar Móveis e Colchões',
                             'datetime': '2019-03-29T18:17:00Z'}],
               'customer': [],
               'final': {}},
 'review_ID': '90143053',
 'tags': ['Estorno do valor pago', 'Mesas e Cadeiras', 'Móveis em

In [3]:
#word_cloud_text_SW = ''.join(without_stop_words)

#wordcloud = WordCloud(max_font_size=100, max_words=100, background_color="white",\
#                           scale = 10,width=800, height=400).generate(word_cloud_text_SW)
#plt.figure()
#plt.imshow(wordcloud, interpolation="bilinear")
#plt.axis("off")
#plt.show()

In [4]:
from datetime import datetime
import numpy as np
import pandas as pd


def extract_days_to_resolution(review):
    if "datetime" in review and "reply" in review["responses"]["final"]:
        init_dt = datetime.strptime(review["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        final_ans_dt = datetime.strptime(review["responses"]["final"]["reply"][0]["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        return (final_ans_dt - init_dt).days
    return np.nan

def extract_days_to_first_contact(review):
    if "datetime" in review and "business" in review["responses"] and len(review["responses"]["business"]) > 0:
        init_dt = datetime.strptime(review["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        final_ans_dt = datetime.strptime(review["responses"]["business"][0]["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        return (final_ans_dt - init_dt).days
    return np.nan

In [5]:
def format_RA_to_df(review):
    cols_for_df = ['title', 'description', 'business_name', 'uf', 'city', 'review_ID', 'datetime', 'timeCaptured']
    r_cp = {col: review[col] for col in cols_for_df if col in review}
    r_cp["days_to_resolution"] = extract_days_to_resolution(review)
    r_cp["days_to_first_contact"] = extract_days_to_first_contact(review)
    r_cp["resolution_outcome"] = review["responses"]["final"]["result"] if "responses" in review and "final" in review["responses"] and "result" in review["responses"]["final"] else np.nan
    return r_cp

dff = pd.DataFrame([format_RA_to_df(row) for row in reviewsRA])
dff.head()

,title,description,business_name,uf,city,review_ID,datetime,timeCaptured,days_to_resolution,days_to_first_contact,resolution_outcome
0,Estorno de entrega,Eu comprei uma mesa no dia 29.01.2019 a vista ...,Inovar Móveis e Colchões,SP,São José dos Campos,90143053,2019-03-28T19:32:00Z,2020-04-02T14:37:21Z,NaN,0.0,NaN
1,Nao entregaram no prazo e descaso não retornam...,Comprei em 26.01 moveis para quarto e mesa sa...,Inovar Móveis e Colchões,SP,São Paulo,90109323,2019-03-27T20:09:00Z,2020-04-02T14:37:24Z,35.0,0.0,positive
2,Insatisfação com o produto,No dia 05/01/2018 fiz uma compra a vista na lo...,Inovar Móveis e Colchões,SP,Jacareí,43153321,2019-02-25T10:59:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN
3,Produto entregue errado e quebrado,Boa tarde! Adquiri uma cama box marron com tec...,Inovar Móveis e Colchões,SP,São José dos Campos,43172911,2019-02-25T18:24:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN
4,"Produto entregue com defeito, aguardando a tro...",Em novembro comprei um sofá na loja inovar de ...,Inovar Móveis e Colchões,SP,São Sebastião,43196029,2019-02-26T12:37:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN


### Count of complaints with response

In [6]:
reviews_with_response = [review for review in reviewsRA if "responses" in  review and len(review["responses"]["business"]) > 0]
reviews_with_no_response = [review for review in reviewsRA if "responses" in  review and len(review["responses"]["business"]) == 0]

print(f"Total of reviews with response {len(reviews_with_response)} [{round((len(reviews_with_response)/len(reviewsRA))*100, 2)}%]")
print(f"Total of reviews with NO response {len(reviews_with_no_response)} [{100- round((len(reviews_with_response)/len(reviewsRA))*100, 2)}%]")
print(f"We have a total of {len(reviewsRA)} reviews!")

Total of reviews with response 7836 [80.94%]
Total of reviews with NO response 1845 [19.060000000000002%]
We have a total of 9681 reviews!


In [7]:
def format_RA_to_df(review):
    cols_for_df = ['title', 'description', 'business_name', 'uf', 'city', 'review_ID', 'datetime', 'timeCaptured']
    r_cp = {col: review[col] for col in cols_for_df if col in review}
    r_cp["days_to_resolution"] = extract_days_to_resolution(review)
    r_cp["days_to_first_contact"] = extract_days_to_first_contact(review)
    r_cp["resolution_outcome"] = review["responses"]["final"]["result"] if "responses" in review and "final" in review["responses"] and "result" in review["responses"]["final"] else np.nan
    return r_cp

dff = pd.DataFrame([format_RA_to_df(row) for row in reviewsRA])
dff.head()

,title,description,business_name,uf,city,review_ID,datetime,timeCaptured,days_to_resolution,days_to_first_contact,resolution_outcome
0,Estorno de entrega,Eu comprei uma mesa no dia 29.01.2019 a vista ...,Inovar Móveis e Colchões,SP,São José dos Campos,90143053,2019-03-28T19:32:00Z,2020-04-02T14:37:21Z,NaN,0.0,NaN
1,Nao entregaram no prazo e descaso não retornam...,Comprei em 26.01 moveis para quarto e mesa sa...,Inovar Móveis e Colchões,SP,São Paulo,90109323,2019-03-27T20:09:00Z,2020-04-02T14:37:24Z,35.0,0.0,positive
2,Insatisfação com o produto,No dia 05/01/2018 fiz uma compra a vista na lo...,Inovar Móveis e Colchões,SP,Jacareí,43153321,2019-02-25T10:59:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN
3,Produto entregue errado e quebrado,Boa tarde! Adquiri uma cama box marron com tec...,Inovar Móveis e Colchões,SP,São José dos Campos,43172911,2019-02-25T18:24:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN
4,"Produto entregue com defeito, aguardando a tro...",Em novembro comprei um sofá na loja inovar de ...,Inovar Móveis e Colchões,SP,São Sebastião,43196029,2019-02-26T12:37:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN


**Correlation between average response time and whether or not a response was positive/negative.**

We lack more data where we have a "negative" resolution, but for those cases where we have a resolution we see that on average that takes one or two days to close.
We might dig deeper to understand what features these kind of complaints have in common.

In [8]:
dff.resolution_outcome = pd.Categorical(dff.resolution_outcome)
dff.groupby("resolution_outcome").mean()

,days_to_resolution,days_to_first_contact
resolution_outcome,,
negative,126.746207,34.064706
positive,49.650507,15.037901


**Average time to company response and evaluating experience.**

On average, companies take 3 days to give the first reply. Also, when aggregating by days before contact, we have a hint that positive responses appear more frequently when the contact is made ealier on.

In [9]:
dff.describe()

,days_to_resolution,days_to_first_contact
count,3489.000000,7836.000000
mean,65.670679,16.201633
std,178.113601,55.441779
min,-1.000000,-1.000000
25%,2.000000,0.000000
50%,10.000000,2.000000
75%,38.000000,12.000000
max,1884.000000,833.000000


In [10]:
dff.groupby("days_to_first_contact").resolution_outcome.count()

days_to_first_contact
-1.0         1
 0.0      1558
 1.0       433
 2.0       262
 3.0       165
          ... 
 659.0       0
 660.0       1
 671.0       1
 687.0       1
 833.0       0
Name: resolution_outcome, Length: 268, dtype: int64

In [11]:
answered_perc = round(dff.days_to_first_contact.count()/dff.shape[0] * 100, 2) 
print(f"Total of answered complaints : {answered_perc}%")

Total of answered complaints : 80.94%


**Percentage of answered complaints by store**

In [12]:
aa = dff.groupby("business_name").count()
aa = round(aa.days_to_first_contact/aa.review_ID * 100, 2) 
pprint(aa)

business_name
                                  91.70
Casas Bahia - Loja Online          1.54
Casas Bahia - Lojas Físicas        6.80
Casas Bahia - Marketplace         12.04
Etna Home Store                   22.22
Etna Home Store - Loja Online     13.04
Fabrispuma                       100.00
Inovar Móveis e Colchões          98.32
Lojas Cem                         55.38
Lojas Guido                       61.11
MadeiraMadeira                    10.80
Magazine Luiza - Loja Física      26.14
Magazine Luiza - Loja Online       0.00
Marabraz - Loja Online            88.01
Marabraz - Lojas Físicas          94.91
Mobly                             41.38
Móveis Rainha                    100.00
Openbox2                          97.30
Sylvia Design                     99.91
TUTTI MÓVEIS                      65.70
Tok&Stok - Loja Física            34.29
Tok&Stok - Loja Online             6.38
Viggore Móveis                   100.00
dtype: float64


**Investigation of ' ' stores**

We see that it must been some sort of bug while scraping data from RA. We have descriptions pointing towards different stores, so it is not a single store problem.

In [13]:
dff.loc[dff["business_name"]=='']

,title,description,business_name,uf,city,review_ID,datetime,timeCaptured,days_to_resolution,days_to_first_contact,resolution_outcome
105,Desconsideração com o consumidor,Infelizmente resolvi compra pela primeira vez ...,,SP,São Paulo,34964387,2018-04-27T22:02:00Z,2020-04-03T15:08:31Z,58.0,5.0,positive
125,Produto não entregue,Boa tarde!Comprei uma cozinha modulada no dia ...,,SP,Itaquaquecetuba,34834923,2018-04-23T17:43:00Z,2020-04-03T15:09:17Z,4.0,0.0,positive
145,Vidro da mesa veio riscado,Comprei um jogo de mesa e 4 cadeiras na loja m...,,SP,Sorocaba,34329727,2018-04-03T23:17:00Z,2020-04-03T15:12:43Z,0.0,0.0,positive
176,Demora na entrega,Bom no dia 05/02/2018 comprei uma cama para me...,,SP,Guarulhos,33866049,2018-03-17T15:54:00Z,2020-04-03T15:15:06Z,17.0,1.0,positive
179,Quero cancelar minha compra pois fui enganada,Fui a uma loja da Inovar no shopping em São J...,,SP,São José dos Campos,30483817,2017-11-21T06:41:00Z,2020-04-02T15:01:37Z,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9386,Até o momento não resolveram meu problema,fazem mais de 15 dias e até agora não vinheram...,,PB,João Pessoa,102019215,2020-03-26T18:18:00Z,2020-03-27T12:18:14Z,NaN,NaN,NaN
9442,comprei um home suspenso grego paguei a vista ...,comprei um home suspenso no dia 24022017 pague...,,SP,Guarulhos,24617837,2017-03-04T15:38:00Z,2020-04-03T18:37:28Z,5.0,3.0,positive
9491,Descaso com o consumidor,Compramos um jogo de estofado de canto que req...,,SP,Salto,93569121,2019-07-24T09:03:00Z,2020-03-30T13:20:05Z,2.0,1.0,positive
9598,Não consigo dormir bem,Comprei uma cama box..A base da cama ficou tod...,,SP,Jacareí,92256803,2019-06-09T02:19:00Z,2020-04-02T13:52:04Z,NaN,1.0,NaN


**Tag analysis**

In [14]:
unique_tags = set()
for review in reviewsRA:
    if "tags" in review:
        for tag in review["tags"]:
            unique_tags.add(tag)

print(f"We have {len(unique_tags)} unique tags!")
pprint(unique_tags)

We have 264 unique tags!
{'Acessórios de Vestuário',
 'Acessórios para Bebê',
 'Acessórios para Carros',
 'Agendamento',
 'Alto falantes',
 'Amassado',
 'Aparadores',
 'Aparelho de jantar',
 'Aparelhos de musculação',
 'Aparelhos de som',
 'Aplicativos',
 'Ar condicionado',
 'Armários de cozinha',
 'Aromatizantes e Difusores',
 'Artigos Religiosos',
 'Artigos para bebê',
 'Aspiradores e vassouras',
 'Assistência técnica',
 'Assistência técnica e Garantia',
 'Assistência técnica indisponível',
 'Atendimento na Loja',
 'Atraso na entrega',
 'Atraso na entrega do produto',
 'Atraso no reembolso por cancelamento',
 'Balanças',
 'Bancos e banquetas',
 'Berço desmontável',
 'Bikes',
 'Bolsas',
 'Bolsas e Malas',
 'Bonecos e Bonecas',
 'Braço quebrado',
 'Braço solto',
 'Brinquedos e jogos',
 'Brinquedos eletrônicos',
 'Brinquedos para bebês',
 'Cabeceira',
 'Cadeira bamba',
 'Cadeiras',
 'Cafeteiras e Chaleiras',
 'Caixas de som e fone de ouvido',
 'Cama, Mesa e Banho',
 'Camas e camas-box',

**What categories of complaints correlate to poor evaluations?**

In [15]:
unique_tags = set()
unique_businesses = set()

for review in reviewsRA:
    if "business_name" in review:
        unique_businesses.add(review['business_name'])
        if "tags" in review:
            for tag in review["tags"]:
                unique_tags.add(tag)

for review in reviewsRA:
    if "tag" in review:
        for tag in review["tags"]:
            unique_tags.add(tag)

print(f"We have {len(unique_businesses)} businesses.")
pprint(unique_businesses)
            
print(f"We have {len(unique_tags)} unique tags.")
pprint(unique_tags)

We have 23 businesses.
{'',
 'Casas Bahia - Loja Online',
 'Casas Bahia - Lojas Físicas',
 'Casas Bahia - Marketplace',
 'Etna Home Store',
 'Etna Home Store - Loja Online',
 'Fabrispuma',
 'Inovar Móveis e Colchões',
 'Lojas Cem',
 'Lojas Guido',
 'MadeiraMadeira',
 'Magazine Luiza - Loja Física',
 'Magazine Luiza - Loja Online',
 'Marabraz - Loja Online',
 'Marabraz - Lojas Físicas',
 'Mobly',
 'Móveis Rainha',
 'Openbox2',
 'Sylvia Design',
 'TUTTI MÓVEIS',
 'Tok&Stok - Loja Física',
 'Tok&Stok - Loja Online ',
 'Viggore Móveis'}
We have 264 unique tags.
{'Acessórios de Vestuário',
 'Acessórios para Bebê',
 'Acessórios para Carros',
 'Agendamento',
 'Alto falantes',
 'Amassado',
 'Aparadores',
 'Aparelho de jantar',
 'Aparelhos de musculação',
 'Aparelhos de som',
 'Aplicativos',
 'Ar condicionado',
 'Armários de cozinha',
 'Aromatizantes e Difusores',
 'Artigos Religiosos',
 'Artigos para bebê',
 'Aspiradores e vassouras',
 'Assistência técnica',
 'Assistência técnica e Garantia',


In [16]:
tags_per_businesses = pd.DataFrame(0, columns = unique_businesses, index = unique_tags)

tags_per_businesses['Mobly']['Eletroportáteis'] += 1

tags_per_businesses.head()

,,Magazine Luiza - Loja Online,Marabraz - Lojas Físicas,Viggore Móveis,Sylvia Design,Casas Bahia - Lojas Físicas,Fabrispuma,Magazine Luiza - Loja Física,Mobly,Lojas Guido,...,Móveis Rainha,Tok&Stok - Loja Online,MadeiraMadeira,Lojas Cem,Tok&Stok - Loja Física,Etna Home Store - Loja Online,Inovar Móveis e Colchões,Marabraz - Loja Online,TUTTI MÓVEIS,Openbox2
Fritadeira elétrica,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Falta de peças,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Climatizador de ar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Equipamentos de rede-Wireless,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Móveis para Jardim e Piscina,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
for review in reviewsRA:
    if "tags" in review:
        for tag in review['tags']:
#            print(review['business_name'])
#            print(tag)
            tags_per_businesses[review['business_name']][tag] += 1 
    
tags_per_businesses

,,Magazine Luiza - Loja Online,Marabraz - Lojas Físicas,Viggore Móveis,Sylvia Design,Casas Bahia - Lojas Físicas,Fabrispuma,Magazine Luiza - Loja Física,Mobly,Lojas Guido,...,Móveis Rainha,Tok&Stok - Loja Online,MadeiraMadeira,Lojas Cem,Tok&Stok - Loja Física,Etna Home Store - Loja Online,Inovar Móveis e Colchões,Marabraz - Loja Online,TUTTI MÓVEIS,Openbox2
Fritadeira elétrica,0,1,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
Falta de peças,0,0,5,0,0,0,0,0,0,0,...,3,0,1,0,0,0,1,5,0,0
Climatizador de ar,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Equipamentos de rede-Wireless,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Móveis para Jardim e Piscina,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Chapinhas (Pranchas),0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Smartphone,0,3,0,0,0,2,0,5,0,0,...,0,0,0,4,0,0,0,0,0,0
Inferior ao vendido,0,0,1,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,0,0,0
Perfume,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
formatted_rows = []

for review in reviewsRA:
    if "business_name" in review and "responses" in review:
        panda_row = {"business_name": review["business_name"]}
        
        if len(review["responses"]["business"]) > 0:
            panda_row["has_reply"] = True
        else:
            panda_row["has_reply"] = False
            
        if "result" in review["responses"]["final"]:
            panda_row["reply_outcome"] = "positive"
        else:
            panda_row["reply_outcome"] = "negative"
        
        panda_row["tags"] = "_".join(review["tags"])
        
        formatted_rows.append(panda_row)

df = pd.DataFrame(formatted_rows)
df.head()

,business_name,has_reply,reply_outcome,tags
0,Inovar Móveis e Colchões,True,negative,Estorno do valor pago_Mesas e Cadeiras_Móveis ...
1,Inovar Móveis e Colchões,True,positive,Atraso na entrega_Móveis_Móveis em Geral
2,Inovar Móveis e Colchões,True,negative,Má qualidade do produto_Sofás e Estofados_Móve...
3,Inovar Móveis e Colchões,True,negative,Colchões
4,Inovar Móveis e Colchões,True,negative,Produto com defeito_Sofás e Estofados_Móveis e...


## NLP Analysis

In [19]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import nltk 
from nltk.corpus import stopwords
#nltk.download('punkt')
#nltk.download('stopwords')
import numpy  as np
import string
#from nltk.stem import PorterStemmer 
import unicodedata

In [20]:
word_cloud_text = ' '.join(dff['description'])

stop_words = set(stopwords.words("portuguese"))
without_stop_words = []
sentence = word_cloud_text
words = nltk.word_tokenize(sentence)
for word in words:
    if word not in stop_words:
        without_stop_words.append(word)

In [21]:
stop_words = set(stopwords.words("portuguese"))
without_stop_words = []
stopword = []
sentence = word_cloud_text
words = nltk.word_tokenize(sentence)
for word in words:
    if word in stop_words:
        stopword.append(word)
    else:
        without_stop_words.append(word)

In [22]:
def clean_reviews(review):
    without_stop_words = []
    words = nltk.word_tokenize(review.lower())
    for word in words:
        if not word in stop_words:
            without_stop_words.append(word)

    return ' '.join(without_stop_words)

In [23]:
def strip_punctuation(text):
    """
    >>> strip_punctuation(u'something')
    u'something'

    >>> strip_punctuation(u'something.,:else really')
    u'somethingelse really'
    """
    punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
    return ''.join(x for x in text
                   if unicodedata.category(x) not in punctutation_cats)

In [24]:
dff['cleaned_review'] = dff.description.apply(lambda x: strip_punctuation(clean_reviews(x)))
    
dff.head()

,title,description,business_name,uf,city,review_ID,datetime,timeCaptured,days_to_resolution,days_to_first_contact,resolution_outcome,cleaned_review
0,Estorno de entrega,Eu comprei uma mesa no dia 29.01.2019 a vista ...,Inovar Móveis e Colchões,SP,São José dos Campos,90143053,2019-03-28T19:32:00Z,2020-04-02T14:37:21Z,NaN,0.0,NaN,comprei mesa dia 29012019 vista prazo entrega ...
1,Nao entregaram no prazo e descaso não retornam...,Comprei em 26.01 moveis para quarto e mesa sa...,Inovar Móveis e Colchões,SP,São Paulo,90109323,2019-03-27T20:09:00Z,2020-04-02T14:37:24Z,35.0,0.0,positive,comprei 2601 moveis quarto mesa sala jantar se...
2,Insatisfação com o produto,No dia 05/01/2018 fiz uma compra a vista na lo...,Inovar Móveis e Colchões,SP,Jacareí,43153321,2019-02-25T10:59:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN,dia 05012018 fiz compra vista loja inovar jaca...
3,Produto entregue errado e quebrado,Boa tarde! Adquiri uma cama box marron com tec...,Inovar Móveis e Colchões,SP,São José dos Campos,43172911,2019-02-25T18:24:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN,boa tarde adquiri cama box marron tecido sued...
4,"Produto entregue com defeito, aguardando a tro...",Em novembro comprei um sofá na loja inovar de ...,Inovar Móveis e Colchões,SP,São Sebastião,43196029,2019-02-26T12:37:00Z,2020-04-02T14:37:46Z,NaN,0.0,NaN,novembro comprei sofá loja inovar sao sebastiã...


In [25]:
from collections import Counter

AllReviews = dff['cleaned_review']

all_reviews_text = ' '.join(AllReviews)
tokenized_words = nltk.word_tokenize(all_reviews_text)
word_freq = Counter(tokenized_words)

word_freq.most_common(20)

[('dia', 14417),
 ('loja', 13062),
 ('entrega', 9157),
 ('dias', 8173),
 ('compra', 7572),
 ('produto', 6937),
 ('contato', 6152),
 ('sofá', 5871),
 ('pois', 5201),
 ('prazo', 4967),
 ('comprei', 4958),
 ('pedido', 4336),
 ('nada', 3839),
 ('empresa', 3423),
 ('marabraz', 3297),
 ('pra', 3149),
 ('problema', 3057),
 ('gerente', 2938),
 ('hoje', 2796),
 ('agora', 2785)]

In [26]:
selected_reviews = [each for each in AllReviews if 'entrega' in each]

for review in selected_reviews[:30]:
    index = review.find('entrega')
    print(review[index-10:index+20].replace("\n", ""))

sta prazo entrega dia 1503 ent
 pergunte entrega informaram e
duto casa entregadores rasgara
s mostrei entregador afirmou p
es  agora entregaram justifica
arcar dia entrega cabeccabecei
mprometeu entregarem todos jun
 realizar entrega moveis 10 di
r produto entregaram  deram ne
té  prazo entrega 0711  portan
 atibaia  entrega deu último d
dia 0508  entrega combinada 30
ro braço  entregar msm detecto
m n feita entrega alegando def
harem bom entrega  entrei cont
visão pra entrega  quero fazer
  agendei entrega  ser prédio 
 disse ia entregar mesa semana
ts porque entregaram porta cor
r disse q entregaria 7 dias an
s  vieram entregar móveis entr
strativas entrega  creio fuja 
edor iria entregar antes natal
ter prazo entrega escolhi agor
o agendou entrega 17h30  chega
ealizaram entrega  dia 2201 ag
problemas entrega  devidamente
rico duas entregas ploblemas  
ra  antes entregar pra gerar m
p agendar entrega pois produto


In [27]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=1,k=1):
    vec = CountVectorizer(ngram_range=(k,k),stop_words = 'portuguese').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [28]:
from nltk.util import ngrams

def top_k_ngrams(word_tokens,n,k):
    
    n_gram_list = list(ngrams(word_tokens, n))

    n_gram_strings = [' '.join(each) for each in n_gram_list]
    
    n_gram_counter = Counter(n_gram_strings)
    most_common_k = n_gram_counter.most_common(k)
    pprint(most_common_k)

In [29]:
port_stopwords = stopwords.words('portuguese')

#n-grams
word_tokens = nltk.word_tokenize(all_reviews_text)
top_k_ngrams(word_tokens, 3, 10)

[('editado reclame aqui', 680),
 ('valor r $', 304),
 ('5 dias úteis', 298),
 ('comprei guarda roupa', 279),
 ('entrei contato sac', 250),
 ('entrei contato loja', 246),
 ('fiz compra dia', 214),
 ('10 dias úteis', 205),
 ('4 dias úteis', 203),
 ('loja sylvia design', 201)]


In [30]:
top_k_ngrams(word_tokens, 2, 10)

[('entrei contato', 1871),
 ('dias úteis', 1749),
 ('fiz compra', 1413),
 ('guarda roupa', 1317),
 ('prazo entrega', 1280),
 ('30 dias', 1125),
 ('r $', 1107),
 ('reclame aqui', 981),
 ('entrega dia', 801),
 ('entrar contato', 784)]


In [31]:
def get_top_n_words(corpus, n=1,k=1):
    vec = CountVectorizer(ngram_range=(k,k),stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [32]:
common_words = get_top_n_words(AllReviews, 20, 2)

for word, freq in common_words:
    print(word, freq)

df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])

entrei contato 1872
dias úteis 1749
fiz compra 1414
guarda roupa 1319
prazo entrega 1280
30 dias 1125
reclame aqui 981
entrega dia 803
entrar contato 784
nota fiscal 720
sylvia design 703
editado reclame 680
10 dias 655
realizei compra 651
falta respeito 643
loja marabraz 632
comprei sofá 627
hoje dia 616
efetuei compra 604
dias uteis 568


In [33]:
#Inovar vs. Sylvia Design

inovar_reviews = [each for each in dff[dff.business_name == 'Inovar Móveis e Colchões'].cleaned_review]

sylvia_reviews = [each for each in dff[dff.business_name == 'Sylvia Design'].cleaned_review]

In [34]:
inovar_reviews_text = ' '.join(inovar_reviews)
tokenized_words = nltk.word_tokenize(inovar_reviews_text)
inovar_word_freq = Counter(tokenized_words)

inovar_word_freq.most_common(20)

[('loja', 2050),
 ('sofá', 1431),
 ('dia', 1394),
 ('entrega', 957),
 ('gerente', 878),
 ('dias', 762),
 ('inovar', 672),
 ('compra', 661),
 ('contato', 637),
 ('produto', 562),
 ('pois', 551),
 ('comprei', 470),
 ('nada', 449),
 ('prazo', 417),
 ('problema', 412),
 ('disse', 411),
 ('vendedor', 393),
 ('troca', 365),
 ('mesa', 333),
 ('empresa', 323)]

In [35]:
sylvia_reviews_text = ' '.join(sylvia_reviews)
tokenized_words = nltk.word_tokenize(sylvia_reviews_text)
sylvia_word_freq = Counter(tokenized_words)

sylvia_word_freq.most_common(20)

[('sofá', 2382),
 ('loja', 2356),
 ('dia', 1871),
 ('entrega', 1256),
 ('dias', 1157),
 ('pois', 1007),
 ('compra', 959),
 ('sylvia', 893),
 ('mesa', 889),
 ('contato', 879),
 ('produto', 862),
 ('design', 759),
 ('prazo', 712),
 ('comprei', 702),
 ('vendedor', 701),
 ('empresa', 662),
 ('sac', 621),
 ('cadeiras', 565),
 ('problema', 565),
 ('disse', 552)]

In [36]:
inovar_word_tokens = nltk.word_tokenize(inovar_reviews_text)
top_k_ngrams(inovar_word_tokens, 4, 10)

[('comprei sofá loja inovar', 15),
 ('center vale josé campos', 13),
 ('inovar shopping center vale', 11),
 ('prazo entrega 30 dias', 11),
 ('loja inovar móveis colchões', 11),
 ('shopping center vale josé', 10),
 ('entrega 30 dias úteis', 9),
 ('compra sofá loja inovar', 9),
 ('loja inovar shopping center', 8),
 ('entrei contato gerente loja', 8)]


In [37]:
sylvia_word_tokens = nltk.word_tokenize(sylvia_reviews_text)
top_k_ngrams(sylvia_word_tokens, 4, 10)

[('sofá loja sylvia design', 29),
 ('entrega 45 60 dias', 18),
 ('loja sylvia design av', 17),
 ('sylvia design raposo tavares', 16),
 ('prazo entrega 45 60', 15),
 ('prazo 45 60 dias', 14),
 ('sylvia design comprei sofá', 14),
 ('comprei sofá sylvia design', 14),
 ('av zaki narchi 42', 14),
 ('loja sylvia design raposo', 14)]


### General impressions
In general, an NLP analysis of the dataset shows that most complaints made against furniture retailers revolve around words related to the order (order, shop, buy, store) or to the actual delivery of the product (day, delivery, days, arrival). A k-gram investigation of the issue confirms our findings and word sets such as "5 working days" and "delivery date" are on top of the list.
We then proceeded to a side-by-side comparison between 2 similar sized businesses: Sylvia Design and Inovar. A word frequency analysis shows us that both experience most customer inquires regarding couches, however, their approaches to delivering their mecharndise differ somewhat. Sylvia Design delivers in 45 to 60 days while Inovar's delivery is 15 days shorter.